In [1]:
# my class
from untrained_encoder import UntrainedAutoencoder

# wilds import
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds import get_dataset

# torchvision transforms
import torchvision.transforms as transforms

# misc
import numpy as np
import os
import json

2023-02-03 08:50:50.054591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-03 08:50:50.054623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/ubuntu/miniconda3/envs/alibitorchwhy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CAMELYON TRAIN LOADING

In [2]:
%cd  '/home/ubuntu/image-drift-monitoring'
%pwd

camelyon_train_comp = np.load('camelyon_train_ds.npz')

camelyon_train = camelyon_train_comp['arr_0']

/home/ubuntu/image-drift-monitoring


In [3]:
camelyon_train.shape

(302436, 3, 96, 96)

shuffling train

In [4]:
import random

np.random.shuffle(camelyon_train)

In [8]:
my_uae = UntrainedAutoencoder()

encoder_fn = my_uae.init_default_tf_encoder(encoding_dim=96,input_shape=camelyon_train.shape[1:],batch_size=512)

# encoder_fn_py = my_uae.init_default_pt_encoder(encoding_dim=96,input_shape=camelyon_train.shape[1:],batch_size=512)

2023-02-03 09:18:21.957793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-03 09:18:21.957848: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-03 09:18:21.957870: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (render): /proc/driver/nvidia/version does not exist
2023-02-03 09:18:21.961420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
len(camelyon_train)

302436

In [6]:
camelyon_train_first = camelyon_train[:int(len(camelyon_train)/2)]

In [7]:
camelyon_train_second = camelyon_train[int(len(camelyon_train)/2):]

In [ ]:
camelyon_train_first.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.transpose(camelyon_train_first[0],(1,2,0)))

In [ ]:
camelyon_train_second.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.transpose(camelyon_train_second[0],(1,2,0)))

INIT KS DETECTOR on first 50% of TRAIN

In [7]:
# my_uae.init_ks_detector(reference_data=camelyon_train_first,encoder_fn=encoder_fn, p_val= 0.05, path='camelyon_uae_ks', save_dec=True)

# my_uae.import_KS_detector('camelyon_uae_ks')

Directory camelyon_uae_ks does not exist and is now created.
Directory camelyon_uae_ks/preprocess_fn/model does not exist and is now created.


2023-02-02 12:51:52,425 [INFO]  KS Detector initialized


INIT MMD DETECTOR on first 50% of TRAIN

In [ ]:
my_uae.init_mmd_detector(reference_data=camelyon_train_first[:int(len(camelyon_train_first)/2)],encoder_fn=encoder_fn,n_permutations=100 ,p_val=0.05,path='/home/ubuntu/image-drift-monitoring/src/modules/alibi_detect/camelyon_uae_ks/camelyon_uae_mmd',save_dec=True)

In [12]:
import torch
import torch.nn as nn
from alibi_detect.cd.pytorch import preprocess_drift
from functools import partial

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define encoder
encoder_net = nn.Sequential(
    nn.Conv2d(3, 64, 4, stride=2, padding=0),
    nn.ReLU(),
    nn.Conv2d(64, 128, 4, stride=2, padding=0),
    nn.ReLU(),
    nn.Conv2d(128, 512, 4, stride=2, padding=0),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(2048, 96)
).to(device).eval()

preprocess_fn = partial(preprocess_drift, model=encoder_net, device=device, batch_size=512)

In [18]:
from alibi_detect.cd import KSDrift, MMDDrift
camelyon_train_first_quant = camelyon_train_first[:int(len(camelyon_train_first)/2)]

camelyon_train_first_quant.shape
detector = MMDDrift(x_ref=camelyon_train_first_quant,backend='pytorch',p_val=0.05,preprocess_fn=preprocess_fn,n_permutations=100)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x51200 and 2048x96)

In [8]:
from alibi_detect.saving import save_detector, load_detector

save_detector(detector=detector,filepath='/home/ubuntu/image-drift-monitoring/src/modules/alibi_detect/camelyon_uae_mmd')

Directory /home/ubuntu/image-drift-monitoring/src/modules/alibi_detect/camelyon_uae_ks/camelyon_uae_mmd does not exist and is now created.
Directory /home/ubuntu/image-drift-monitoring/src/modules/alibi_detect/camelyon_uae_ks/camelyon_uae_mmd/preprocess_fn/model does not exist and is now created.


MAKE KS PREDICTION ON SECOND 50% of TRAIN

In [9]:
len(camelyon_train_first_quant)

37804

In [ ]:
my_uae.make_prediction(target_data=camelyon_train_second,type='KS')

MAKE MMD PREDICTION ON SECOND 50% of TRAIN

In [ ]:
my_uae.make_prediction(target_data=camelyon_train_second,type='MMD')